In [1]:
# Core
import numpy as np
import pandas as pd
import warnings

# Machine Learning
from sklearn import metrics, preprocessing, tree
from sklearn.metrics import  f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [2]:
# Evitar los warnings
warnings.filterwarnings('ignore')

In [3]:
# Dataset
data = pd.read_csv('https://raw.githubusercontent.com/jorge-robledo11/Datasets/main/Datasets/Social_Network_Ads.csv')
data.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
# Dimensiones
data.shape

(400, 5)

In [5]:
"""
1. Importamos nuestro estimador
2. Definimos nuestros features y target
"""

from sklearn.tree import DecisionTreeClassifier

X = data.iloc[:,[2,3]].values
y = data.iloc[:,4].values

In [6]:
# Separando y entrenado los subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

## Cross Validation

In [8]:
"""
1. Definimos el clasificador e instanciamos
2. Hacemos validación cruzada y sacamos la media de los resultados
"""

clf = DecisionTreeClassifier()

scores = cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1, scoring='f1_macro').mean()
scores

0.817141371470228

## Grid Search

In [9]:
# Obtenemos nuestra grilla de parámetros
parameters = {'max_depth': [1,2,3,4,5],
              'min_samples_leaf': [1,2,3,4,5],
              'min_samples_split': [2,3,4,5],
              'criterion': ['gini','entropy']}

In [10]:
"""
1. Entrenamos nuestro clasificador con validación cruzada y los parámetros conseguidos
2. Printeamos los resultados de las medias de las validaciones cruzadas de todas las combinaciones posibles de modelos 
con los parámetros
3. Tomamos el mejor modelo de los resultados obtenidos
"""

clf_rcv = RandomizedSearchCV(clf, parameters, cv=5, n_jobs=-1, scoring='f1_macro').fit(X_train, y_train)
print(clf_rcv.cv_results_['mean_test_score'])
best_model_r = clf_rcv.best_estimator_

[0.84507158 0.89566755 0.89599762 0.89599762 0.78915787 0.85727076
 0.89599762 0.78915787 0.86553328 0.8397691 ]


In [11]:
# Hacemos validación cruzada con nuestro mejor modelo y sacamos la media de la mejor validación cruzada
scores = cross_val_score(best_model_r, X_train, y_train, cv=5, scoring='f1_macro').mean()
scores = round(scores,3)
scores

0.896

In [12]:
# Entrenamos el modelo con los datos de entrenamiento y printeamos el score con los resultados tanto de train como de test

best_model_r.fit(X_train, y_train)
print(round(best_model_r.score(X_train, y_train),3))
print(round(best_model_r.score(X_test, y_test),3))

0.909
0.925


## Hyperparameters Tuning

In [13]:
"""
1. Entrenamos nuestro clasificador con validación cruzada y los parámetros conseguidos
2. Printeamos los resultados de las medias de las validaciones cruzadas de todas las combinaciones posibles de modelos 
con los parámetros
3. Tomamos el mejor modelo de los resultados obtenidos
"""

clf_cv = GridSearchCV(clf, parameters, cv=5, n_jobs=-1, scoring='f1_macro').fit(X_train, y_train)
print(clf_cv.cv_results_['mean_test_score'])
best_model = clf_cv.best_estimator_

[0.78915787 0.78915787 0.78915787 0.78915787 0.78915787 0.78915787
 0.78915787 0.78915787 0.78915787 0.78915787 0.78915787 0.78915787
 0.78915787 0.78915787 0.78915787 0.78915787 0.78915787 0.78915787
 0.78915787 0.78915787 0.89599762 0.89599762 0.89599762 0.89599762
 0.89599762 0.89599762 0.89599762 0.89599762 0.89599762 0.89599762
 0.89599762 0.89599762 0.89599762 0.89599762 0.89599762 0.89599762
 0.89599762 0.89599762 0.89599762 0.89599762 0.88548069 0.88548069
 0.88548069 0.88548069 0.88209749 0.88209749 0.88209749 0.88209749
 0.88548069 0.88548069 0.88548069 0.88548069 0.89566755 0.89566755
 0.89566755 0.89566755 0.89566755 0.89566755 0.89566755 0.89566755
 0.83936213 0.84356318 0.84356318 0.84356318 0.84713676 0.84713676
 0.84713676 0.84713676 0.85417767 0.85417767 0.85417767 0.85417767
 0.86553328 0.86553328 0.86553328 0.86553328 0.86553328 0.86553328
 0.86553328 0.86553328 0.84891384 0.84090015 0.84496774 0.84496774
 0.83700241 0.84071137 0.83664378 0.84071137 0.84081787 0.8408

In [14]:
# Hacemos validación cruzada con nuestro mejor modelo y sacamos la media de la mejor validación cruzada

scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='f1_macro').mean()
scores = round(scores,3)
scores

0.896

In [15]:
# Entrenamos el modelo con los datos de entrenamiento y printeamos el score con los resultados tanto de train como de test
best_model.fit(X_train, y_train)
print(round(best_model.score(X_train, y_train),3))
print(round(best_model.score(X_test, y_test),3))

0.909
0.925


In [16]:
# Observamos los mejores parámetros del estimador
print(clf_cv.best_params_)

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [17]:
# Definimos nuestro segundo clasificador
clf2 = DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=3, min_samples_split=2)

clf2.fit(X_train, y_train)
print(round(clf2.score(X_train, y_train),3))
print(round(clf2.score(X_test, y_test),3))

0.916
0.938


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4ec84517-f553-446b-9032-1da3132bd62a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>